In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

OUTPUT_DIR = "../data/model_output"

folder_paths = [OUTPUT_DIR]

# Create the output directories if they doesn't exist
for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [2]:
import albumentations as A
from plot import get_bounding_boxes, create_bboxes_image
import cv2
import numpy as np
from layers import AMF_GD_YOLOv8

### Use the AMF GD YOLOv8 model

In [3]:
class_names = {0: "Tree", 1: "Tree_disappeared", 2: "Tree_replaced", 3: "Tree_new"}

model = AMF_GD_YOLOv8(3, 1, scale="s", class_names=class_names)

rgb_image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_0_1.tif"
chm_image_path = (
    "../data/CHM_cropped_0p24/122000_484000/122000_484000_0_1_filtered_chm.tif"
)

output_name = "Model_output_test.jpg"
output_path = os.path.join(OUTPUT_DIR, output_name)

output = model(rgb_image_path, chm_image_path, image_save_path=output_path)

### How to use data augmentation with Albumentations

In [33]:
from time import time

transform = A.Compose(
    [
        A.RandomCrop(width=640, height=640, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.RandomBrightnessContrast(p=1.0),
        A.GridDistortion(
            num_steps=10,
            distort_limit=0.1,
            border_mode=cv2.BORDER_CONSTANT,
            normalized=False,
            p=0.25,
        ),
        A.Perspective(p=0.25, interpolation=cv2.INTER_LINEAR),
    ],
    bbox_params=A.BboxParams(
        format="coco", min_area=0, min_visibility=0.2, label_fields=["class_labels"]
    ),
)

# image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# annotations_path = "../data/annotations_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.json"
image_path = "../data/images_full/2023_122000_484000_RGB_hrl.tif"
annotations_path = "../data/annotations_full/3_all_annotations.json"

t = time()

image = cv2.imread(image_path)
bboxes, labels = get_bounding_boxes(annotations_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 10

colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

t = time()

for i in range(number_tests):
    transformed = transform(image=image, bboxes=bboxes, class_labels=labels)
    transformed_image = transformed["image"]
    transformed_bboxes = transformed["bboxes"]
    transformed_class_labels = transformed["class_labels"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    bboxes_image = create_bboxes_image(
        transformed_image,
        transformed_bboxes,
        labels=transformed_class_labels,
        colors_dict=colors,
        scores=np.random.rand((len(bboxes))),
    )

    output_name = f"Augmentation_test_{i}_bboxes.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, bboxes_image)

print(f"Time elapsed: {round(time() - t, 3)}")

Time elapsed: 1.703
Time elapsed: 0.887
